In [3]:
# Setup:
# https://towardsdatascience.com/step-by-step-tutorial-pyspark-sentiment-analysis-on-google-dataproc-fef9bef46468 
# and https://cloud.google.com/dataproc/docs/tutorials/jupyter-notebook?authuser=1#console 

# Import all libraries needed for fitting ALS collbaorative filtering model on SQL dataframe
import pandas as pd
from pyspark.sql import SparkSession
# for working with sql dataframes
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from time import time


In [8]:
# create a SparkSession
sparkSess = SparkSession.builder.appName('movieApp').getOrCreate()
# read the data with schema
# data = sparkSess.read.csv('./ml-latest-small/ratings.csv',inferSchema=True,header=True)
data = sparkSess.read.csv('gs://bucket-capstoneudacitydatascience/ml-latest/ratings.csv',inferSchema=True,header=True)
sparkSess.sparkContext.defaultParallelism, data.rdd.getNumPartitions()

(3, 6)

In [9]:
########### Training and validation of model #############
##########################################################

# a range for the number of latent factors in ALS model 
ranks = [4, 8, 12]
# value of regularization
reg_Param = 0.01
# maximum nuber of iterations for training 
max_Iter = 5

# split data in trainig testing and validation subsets
(training, validation, test) = data.randomSplit([0.6, 0.2, 0.2])
# intialize fixed part of ALS model
als = ALS(maxIter=max_Iter, regParam=reg_Param, userCol="userId", itemCol="movieId", ratingCol="rating")
# define RMSE evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

min_error = float('inf')
best_rank = -1
for rank in ranks:
    # Set the rank here
    als.setRank(rank)
    # record training time 
    t0 = time()
    # fit the model with these parameters
    model = als.fit(training)
    tt = time() - t0
    print ("Training completed in {} seconds".format(round(tt,3)))    
    
    # create rating predictions against the validation dataframe
    predict_df_val = model.transform(validation)
    # remove NaN values from predictions 
    predicted_ratings_df_val = predict_df_val.filter(predict_df_val.prediction != float('nan'))
    # evaluate rmse for predicted ratings
    error = evaluator.evaluate(predicted_ratings_df_val)

    print ('For rank %s, the validation RMSE is %s' % (rank, error))
    # record the best rank
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)
# fit the model for best rank
als.setRank(best_rank)
t0 = time()
model = als.fit(training)
tt = time() - t0
print ("Training completed in {} seconds".format(round(tt,3)))    

# compute rmse for testing dataset 
predict_df = model.transform(test)
predicted_test_df = predict_df.filter(predict_df.prediction != float('nan'))
test_RMSE = evaluator.evaluate(predicted_test_df)
print('The model had a RMSE on the test set of {0}'.format(test_RMSE))

Training completed in 75.073 seconds
For rank 4, the validation RMSE is 0.85267451103
Training completed in 70.074 seconds
For rank 8, the validation RMSE is 0.860100462408
Training completed in 76.208 seconds
For rank 12, the validation RMSE is 0.859417291424
The best model was trained with rank 4
Training completed in 59.286 seconds
The model had a RMSE on the test set of 0.852109611708


In [10]:
# try creating recommendation for a user 
single_user = test.filter(test['userId']==11).select(['movieId','userId'])
t0 = time()
reccomendations = model.transform(single_user)
tt = time() - t0
print ("Prediction completed in {} seconds".format(round(tt,3)))    
reccomendations.orderBy('prediction',ascending=False).show()


Prediction completed in 0.038 seconds
+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|   2502|    11|  4.146366|
|   2791|    11| 3.9784715|
|   1282|    11| 3.6588442|
+-------+------+----------+



In [13]:
# Use SparkContext to create resilient distributed dataset (RDD) 
# ratings_raw_data = sparkSess.sparkContext.textFile('./ml-latest-small/ratings.csv')
ratings_raw_data = sparkSess.sparkContext.textFile('gs://bucket-capstoneudacitydatascience/ml-latest/ratings.csv',10)
# re-format rdd for usage with ALS 
ratings_raw_data_header = ratings_raw_data.take(1)[0]
# remove header and separate (userID, movieID, rating)
ratings_rdd = ratings_raw_data.filter(lambda line: line!=ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
sparkSess.sparkContext.defaultParallelism, data.rdd.getNumPartitions()

(3, 6)

In [14]:
# import MLLib version of ALS 
from pyspark.mllib.recommendation import ALS as ALS2
import math

# split data in trainig testing and validation subsets
training_ratings_rdd, validation_ratings_rdd, test_ratings_rdd = ratings_rdd.randomSplit([6, 2, 2], seed=0)
# remove ratings from validation and testing datasets
validation_for_predict_ratings_rdd = validation_ratings_rdd.map(lambda x: (x[0], x[1]))
test_for_predict_ratings_rdd = test_ratings_rdd.map(lambda x: (x[0], x[1]))

min_error = float('inf')
best_rank = -1
for rank in ranks:
    # record training time 
    t0rdd = time()
    # fit the model with these parameters
    model2 = ALS2.train(training_ratings_rdd, rank, iterations=max_Iter,
                      lambda_=reg_Param)
    ttrdd = time() - t0rdd
    print ("Training completed in {} seconds".format(round(ttrdd,3)))

    # create rating predictions against the validation dataframe
    predictions_val = model2.predictAll(validation_for_predict_ratings_rdd).map(lambda r: ((r[0], r[1]), r[2]))
    # join ratings and predictions to compute rmse
    rates_and_preds_val = validation_ratings_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions_val)
    # evaluate rmse for predicted ratings
    error = math.sqrt(rates_and_preds_val.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    print ('For rank %s, the validation RMSE is %s' % (rank, error))
    # record the best rank
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)
# fit the model for best rank
t0rdd = time()
model2 = ALS2.train(training_ratings_rdd, 4, iterations=max_Iter,lambda_=reg_Param)
ttrdd = time() - t0rdd
print ("Training completed in {} seconds".format(round(ttrdd,3)))

# compute rmse for testing dataset 
predictions_test = model2.predictAll(test_for_predict_ratings_rdd).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds_test = test_ratings_rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions_test)
error = math.sqrt(rates_and_preds_test.map(lambda r: (r[1][0] - r[1][1])**2).mean())
print ('For testing data, the RMSE is %s' % (error))

Training completed in 166.108 seconds
For rank 4, the validation RMSE is 0.85985109078
Training completed in 91.894 seconds
For rank 8, the validation RMSE is 0.86134568973
Training completed in 108.552 seconds
For rank 12, the validation RMSE is 0.860909679124
The best model was trained with rank 4
Training completed in 92.444 seconds
For testing data, the RMSE is 0.858461435582


In [15]:
# try creating recommendation for a user 
single_user = test_ratings_rdd.filter(lambda line: line[0]=='11').map(lambda x: (x[0], x[1]))
test_ratings_rdd.take(2),test_ratings_rdd.count(),single_user.count()
t0rdd = time()
reccomendations = model2.predictAll(single_user)
ttrdd = time() - t0rdd
print ("Prediction completed in {} seconds".format(round(ttrdd,3)))
reccomendations.take(10)

Prediction completed in 14.22 seconds


[Rating(user=11, product=2502, rating=4.246173387283797),
 Rating(user=11, product=3623, rating=2.532487153760588)]